## Minimal example of code crash with large offset observations


When creating excess maps with IACTBasicImageEstimator and the RingBackgroundEstimator, the code crashes when using large offset observations (>3.5°) and low energies (<200 GeV) in the spline of the PSF. This might be due to the fact that the PSF is not defined (zero values) in the IRFs for this range of parameters.

In [1]:
%matplotlib inline
import sys
sys.path.append("/Users/facero/Documents/Work/Program/gammapy/gammapy")



import astropy.units as u
from astropy.coordinates import SkyCoord, Angle
from regions import CircleSkyRegion

from gammapy.data import DataStore
from gammapy.image import SkyImage, IACTBasicImageEstimator
from gammapy.background import RingBackgroundEstimator


DATA_DIR = '$CTADATA/index/gps'
ds = DataStore.from_dir(DATA_DIR)


In [2]:
ras = ds.obs_table['RA_PNT'] * u.deg
decs = ds.obs_table['DEC_PNT'] * u.deg

target_position = SkyCoord(347.3, -0.5, unit='deg', frame='galactic')
pointings = SkyCoord(ras, decs, frame='icrs' )
dists = pointings.separation(target_position)
obs_id = ds.obs_table['OBS_ID'][(dists < 4.5 * u.deg) & (dists > 4 * u.deg)]
print(len(obs_id), "observations within radius")

obs_list = ds.obs_list(obs_id)


19 observations within radius


In [3]:
# Define reference image centered on the target
xref = target_position.galactic.l.value
yref = target_position.galactic.b.value
size = 5 * u.deg
binsz = 0.025 # degree per pixel
npix = int((size / binsz).value)

ref_image = SkyImage.empty(
    nxpix=npix, nypix=npix, binsz=binsz,
    xref=xref, yref=yref,
    proj='TAN', coordsys='GAL',
)


on_region = CircleSkyRegion(center=target_position, radius=0.1 * u.deg)

exclusion_mask = ref_image.region_mask(on_region)
exclusion_mask.data = 1 - exclusion_mask.data

# Code crash

The following cell will crash when querying energy ranges that are outside the IRF PSF definition.
For E<0.1 TeV, the code will crash as PSF is not defined in the IRFs.

Changing emin to 0.3 TeV avoids the crash.

In [4]:
bkg_estimator = RingBackgroundEstimator(
    r_in=0.8 * u.deg,
    width=0.2 * u.deg)

image_estimator = IACTBasicImageEstimator(
    reference=ref_image,
    emin=0.05 * u.TeV,
    emax=1 * u.TeV,
    offset_max=3 * u.deg,
    background_estimator=bkg_estimator,
    exclusion_mask=exclusion_mask)

images = image_estimator.run(obs_list)
images.names

/Users/facero/Documents/Work/Program/gammapy/gammapy/gammapy/cube/core.py:85: RuntimeWarning: divide by zero encountered in log
  log_data = np.log(self.data.value)
/Users/facero/Documents/Work/Program/Anaconda/envs/cta/lib/python3.6/site-packages/scipy/interpolate/interpolate.py:2444: RuntimeWarning: invalid value encountered in multiply
  values += np.asarray(self.values[edge_indices]) * weight[vslice]
/Users/facero/Documents/Work/Program/gammapy/gammapy/gammapy/background/ring.py:321: RuntimeWarning: invalid value encountered in true_divide
  result['alpha'] = SkyImage(data=exposure_on.data / result['exposure_off'].data, wcs=wcs)
/Users/facero/Documents/Work/Program/gammapy/gammapy/gammapy/image/models/gauss.py:254: RuntimeWarning: invalid value encountered in true_divide
  self.norms /= self.integral
/Users/facero/Documents/Work/Program/gammapy/gammapy/gammapy/irf/psf_table.py:382: RuntimeWarning: invalid value encountered in less_equal
  i = (np.diff(y) <= 0).argmax()


error: (xb<=x[0]) failed for 2nd keyword xb: fpcurf0:xb=nan